In [1]:
import requests
import pandas as pd
import base64
import gzip
from io import BytesIO
import io
import shutil
from datetime import datetime, timedelta, date
import time
import os
import logging
import smtplib
import pyodbc

CD2_base_url = os.environ['CD2_base_url']
CD2_client_id = os.environ['CD2_client_id']
CD2_client_secret = os.environ['CD2_client_secret']



In [2]:
def les_access_token(logger):
    # Hent access_token
    requesturl = "https://api-gateway.instructure.com/ids/auth/login"
    payload = {'grant_type': 'client_credentials'}
    r = requests.request(
        "POST",
        requesturl,
        data=payload,
        auth=(CD2_client_id, CD2_client_secret)
    )
    if r.status_code == 200:
        respons = r.json()
        access_token = respons['access_token']
        logger.info(f"Henta access_token OK: {access_token}")
        return access_token
    else:
        logger.error(f"Klarte ikkje å skaffe access_token, feil {r.status_code}")
        return None

In [3]:
def lag_logger(log_namn):
    # opprett ein logger
    logger = logging.getLogger('my_logger')
    logger.setLevel(logging.DEBUG)  # Sett ønska loggnivå

    # Opprett formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    # Opprett filhandler for å logge til fil (ein loggfil kvar dag)
    file_handler = logging.FileHandler(log_namn)
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(formatter)

    # Opprett konsollhandler for å logge til konsollen
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.DEBUG)
    console_handler.setFormatter(formatter)

    # Legg til handlerne i loggeren
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    return logger

In [4]:
def hent_CD2_filar(innfil, token, svar, logger):
    try:
        requesturl = f"{CD2_base_url}/dap/object/url"
        payload = f"{svar['objects']}"
        payload = payload.replace('\'', '\"')
        headers = {'x-instauth': token, 'Content-Type': 'text/plain'}
        logger.info(f"Request: {requesturl} {payload}")
        respons = requests.request("POST", requesturl, headers=headers, data=payload)
        logger.info(f"Response: {respons.status_code} {respons.reason}")
        respons.raise_for_status()
        fil = respons.json()
        logger.info(f"Objects: {fil}")
        url = fil['urls'][innfil]['url']
        logger.info(f"URL: {url}")
        data = requests.request("GET", url)
        logger.info(f"Response: {data.status_code} {data.reason}")
        buffer = io.BytesIO(data.content)
        logger.info(f"Buffer: {buffer}")
        with gzip.GzipFile(fileobj=buffer, mode='rb') as utpakka_fil:
            utpakka_data = utpakka_fil.read().decode("utf-8", errors='ignore')
            # logger.info(f"Data: {utpakka_data}")
        return utpakka_data
    except requests.exceptions.RequestException as exc:
        raise exc

In [5]:
def les_CD2_tabell(token, tabell, logger):
    headers = {'x-instauth': token, 'Content-Type': 'text/plain'}
    # sist_oppdatert = akv_finn_sist_oppdatert(tabell)
    payload = '{"format": "csv"}' # % (sist_oppdatert)
    requesturl = f"{CD2_base_url}/dap/query/canvas/table/{tabell}/data"
    print(f"Sender søk til {requesturl}")
    try:
        r = requests.request("POST", requesturl, headers=headers, data=payload)
        r.raise_for_status()
        respons = r.json()
        id = respons['id']
        vent = True
        while vent:
            requesturl2 = f"{CD2_base_url}/dap//job/{id}"
            r2 = requests.request("GET", requesturl2, headers=headers)
            time.sleep(5)
            respons2 = r2.json()
            print(respons2)
            if respons2['status'] == "complete":
                vent = False
                filar = respons2['objects']
        dr_liste = []
        print(filar)
        for fil in filar:
            data = io.StringIO(akv_hent_CD2_filar(fil['id'], token, respons2))
            df = pd.read_csv(data, sep=",")
            dr_liste.append(df)
        alledata = pd.concat(df for df in dr_liste if not df.empty)
        return alledata, sist_oppdatert, respons2['until']
    except requests.exceptions.RequestException as exc:
        raise exc

In [6]:

def akv_finn_sist_oppdatert(tabell):
    """
    Return the latest update time for the given table from the akv_sist_oppdatert table.
    """
    conn_str = os.environ["Connection_SQL"] 
    try:
        with pyodbc.connect(conn_str) as connection:
            cursor = connection.cursor()
            print(connection)
            query = """
            SELECT [sist_oppdatert] FROM [dbo].[akv_sist_oppdatert]
            WHERE [tabell] = ?
            """
            cursor.execute(query, (tabell,))
            row = cursor.fetchone()
            print(row)
            if row:
                print("Har henta frå Azure")
                if tabell == "web_logs":
                    return (datetime.now() - timedelta(days=1)).isoformat() + "Z"
                else:
                    return row[0].isoformat() + "Z"
            else:
                print("Har ikkje henta frå Azure")
                if tabell == "web_logs":
                    return (datetime.now() - timedelta(days=1)).isoformat() + "Z"
                else:
                    return (date.today() - timedelta(days=1)).isoformat() + "Z"
    except pyodbc.Error as exc:
        print("Har ikkje henta frå Azure")
        if tabell == "web_logs":
            return (datetime.now() - timedelta(days=1)).isoformat() + "Z"
        else:
            return (datetime.today() - timedelta(days=1)).isoformat() + "Z"

In [21]:
tabell = "enrollments"
logger = lag_logger(f'loggfil-{tabell}.log')
token = les_access_token(logger)
# data = les_CD2_tabell(token, tabell, logger)
headers = {'x-instauth': token, 'Content-Type': 'text/plain'}
sist_oppdatert = akv_finn_sist_oppdatert(tabell)
sist_oppdatert = "2025-01-01T01:00:00Z"


2025-02-21 14:55:51,770 - my_logger - INFO - Henta access_token OK: eyJhbGciOiJSUzI1NiIsImtpZCI6InB1YmxpYzpoeWRyYS5qd3QuYWNjZXNzLXRva2VuIiwidHlwIjoiSldUIn0.eyJhdWQiOltdLCJjbGllbnRfaWQiOiJldS13ZXN0LTEjMTUwNWNjNGUtZjlhYS00MWQ1LWEzNjQtMjE0Njk5ZWJlNjZiIiwiZXhwIjoxNzQwMTQ5NzUxLCJleHQiOnt9LCJpYXQiOjE3NDAxNDYxNTEsImlzcyI6Imh0dHBzOi8vaXNzLWV1LXdlc3QtMS5pZGVudGl0eS5pbnN0cnVjdHVyZS5jb20vIiwianRpIjoiYzkxMjA4ZDEtMjJmNi00OWMzLTkwYTItZTE1MzQ3OTY3OTNhIiwibmJmIjoxNzQwMTQ2MTUxLCJzY3AiOlsiZGFwIiwicGFydG5lcjpJTlNUSVRVVElPTkFMX1BBUlRORVIiLCJyZWdpb246ZXUtd2VzdC0xIiwicHJpbmNpcGFsOktqWVhGbFd5cUlQMzA2ak1Mck94T0U2UVNVMVM4b0Y1bTV0ZmJYeWQiXSwic3ViIjoiZXUtd2VzdC0xIzE1MDVjYzRlLWY5YWEtNDFkNS1hMzY0LTIxNDY5OWViZTY2YiJ9.A2-UPNjGTr4Ds7TlPi8bhInUItflRcH44pK7x7rKijcFnpZHF8bnRrCibpYl6HB7z-chb7_TrYDGOHvOsgYmHSL5j3U5kf9VrfEFNEnCcYlsSUpd4NbQM7e24h-Be69qQtEoHz2VMTPbGtQgtZ93ox1CDOCAFG9VKIxZD0hiXnRZXwXTEYoBqXuWRmGhJJ2JVWJfqMM7RSYK8946cnUBsFXNmzUIACNjWECauxoVQI-RbtsHARyJFXBtA-S2VhBdm0xm1dov611npUUafBfUe2WBZEj-qII3R0ZkRVT56ub1-fd

Har ikkje henta frå Azure


In [22]:
payload = '{"format": "csv", "since": \"%s\"}' % (sist_oppdatert)
requesturl = f"{CD2_base_url}/dap/query/canvas/table/{tabell}/data"
print(f"Sender søk til {requesturl}")
# try:
r = requests.request("POST", requesturl, headers=headers, data=payload)
r.raise_for_status()
respons = r.json()
id = respons['id']
vent = True
while vent:
    requesturl2 = f"{CD2_base_url}/dap//job/{id}"
    r2 = requests.request("GET", requesturl2, headers=headers)
    time.sleep(5)
    respons2 = r2.json()
    print(respons2)
    if respons2['status'] == "complete":
        vent = False
        filar = respons2['objects']
dr_liste = []
print(filar)


Sender søk til https://api-gateway.instructure.com/dap/query/canvas/table/enrollments/data
{'id': '14207ab8-0144-41de-9203-0250b5d88861', 'status': 'running', 'expires_at': '2025-02-22T13:55:54Z'}
{'id': '14207ab8-0144-41de-9203-0250b5d88861', 'status': 'running', 'expires_at': '2025-02-22T13:55:54Z'}
{'id': '14207ab8-0144-41de-9203-0250b5d88861', 'status': 'running', 'expires_at': '2025-02-22T13:55:54Z'}
{'id': '14207ab8-0144-41de-9203-0250b5d88861', 'status': 'running', 'expires_at': '2025-02-22T13:55:54Z'}
{'id': '14207ab8-0144-41de-9203-0250b5d88861', 'status': 'running', 'expires_at': '2025-02-22T13:55:54Z'}
{'id': '14207ab8-0144-41de-9203-0250b5d88861', 'status': 'running', 'expires_at': '2025-02-22T13:55:54Z'}
{'id': '14207ab8-0144-41de-9203-0250b5d88861', 'status': 'running', 'expires_at': '2025-02-22T13:55:54Z'}
{'id': '14207ab8-0144-41de-9203-0250b5d88861', 'status': 'running', 'expires_at': '2025-02-22T13:55:54Z'}
{'id': '14207ab8-0144-41de-9203-0250b5d88861', 'status': 'run

In [23]:
for fil in filar:
    logger.info(f"Henter fil {fil['id']}")
    data = io.StringIO(hent_CD2_filar(fil['id'], token, respons2, logger))
    df = pd.read_csv(data, sep=",")
    dr_liste.append(df)
alledata = pd.concat(df for df in dr_liste if not df.empty)

2025-02-21 15:08:25,013 - my_logger - INFO - Henter fil 14207ab8-0144-41de-9203-0250b5d88861/part-00000-dd12088c-69c0-405a-921d-2f61d1e4db5c-c000.csv.gz
2025-02-21 15:08:25,013 - my_logger - INFO - Henter fil 14207ab8-0144-41de-9203-0250b5d88861/part-00000-dd12088c-69c0-405a-921d-2f61d1e4db5c-c000.csv.gz
2025-02-21 15:08:25,022 - my_logger - INFO - Request: https://api-gateway.instructure.com/dap/object/url [{"id": "14207ab8-0144-41de-9203-0250b5d88861/part-00000-dd12088c-69c0-405a-921d-2f61d1e4db5c-c000.csv.gz"}, {"id": "14207ab8-0144-41de-9203-0250b5d88861/part-00002-dd12088c-69c0-405a-921d-2f61d1e4db5c-c000.csv.gz"}]
2025-02-21 15:08:25,022 - my_logger - INFO - Request: https://api-gateway.instructure.com/dap/object/url [{"id": "14207ab8-0144-41de-9203-0250b5d88861/part-00000-dd12088c-69c0-405a-921d-2f61d1e4db5c-c000.csv.gz"}, {"id": "14207ab8-0144-41de-9203-0250b5d88861/part-00002-dd12088c-69c0-405a-921d-2f61d1e4db5c-c000.csv.gz"}]


2025-02-21 15:08:25,788 - my_logger - INFO - Response: 200 OK
2025-02-21 15:08:25,788 - my_logger - INFO - Response: 200 OK
2025-02-21 15:08:25,796 - my_logger - INFO - Objects: {'urls': {'14207ab8-0144-41de-9203-0250b5d88861/part-00002-dd12088c-69c0-405a-921d-2f61d1e4db5c-c000.csv.gz': {'url': 'https://data-access-platform-output-prod-dub.s3.eu-west-1.amazonaws.com/output/rootAccountId%3DKjYXFlWyqIP306jMLrOxOE6QSU1S8oF5m5tfbXyd/queryId%3D14207ab8-0144-41de-9203-0250b5d88861/part-00002-dd12088c-69c0-405a-921d-2f61d1e4db5c-c000.csv.gz?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEK7%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCWV1LXdlc3QtMSJIMEYCIQDXmsx%2BVgr5HZMrNRcMjS1bzY1D29yOPAtt8QZRleNEnQIhAKXWiqa3EK4jJ5UodEWaNcKx9lvpzv29glFZ4gdt7ltHKq0CCNf%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQAxoMNTMyMjA1MzAxMzM0Igz37dW0R95ur%2BXF2gQqgQI5IslHiAMHr4KiEM5e1jsKma0YiyEy8FI3O63ALrIa6qRUPNrUI9zFwV%2Fwq2aQ2ttrqFYHAxMfdBIVXFaPetLdeghy5J0apMnNzt3WqyryCt5xP72vYpvZ67xr0dTsgamwZkWMeygzUgf9Zx7qCGTGtyD3TRXNmlDeUB3neBJt7NV6egmA4aEHw%2Fx

In [24]:
alledata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302226 entries, 0 to 302225
Data columns (total 23 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   key.id                                    302226 non-null  int64  
 1   value.sis_batch_id                        284864 non-null  float64
 2   value.user_id                             302226 non-null  int64  
 3   value.created_at                          302226 non-null  object 
 4   value.updated_at                          302226 non-null  object 
 5   value.workflow_state                      302226 non-null  object 
 6   value.role_id                             302226 non-null  int64  
 7   value.start_at                            0 non-null       float64
 8   value.end_at                              0 non-null       float64
 9   value.course_id                           302226 non-null  int64  
 10  value.completed_at  

In [29]:
alledata[['value.user_id', 'value.course_id', 'value.last_activity_at']].to_csv("brukarar_last_login.csv", index=False)